In [1]:
# Data processing
import numpy as np
import pandas as pd

# Download data
import yfinance as yf

In [2]:
# Initial investment and currency per stock

investment = {
    "^VIX" : [1000000, 'USD'],
    "^DJI" : [2500000, 'USD'],
    "^AEX"  : [500000, 'EUR'],
    "^GSPC" : [2300000, 'USD'],
    "^N225" : [1750000, 'JPY']
}

start = "2010-01-01"
end = "2018-12-31"

In [3]:
# Download data and store in dataframe

stockprice_data = yf.download(' '.join(list(investment.keys())), start=start, end=end, groupby='ticker')
stockprice = stockprice_data['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [4]:
stockprice.tail()

,^AEX,^DJI,^GSPC,^N225,^VIX
Date,,,,,
2018-12-24,478.339996,21792.199219,2351.100098,NaN,36.070000
2018-12-25,NaN,NaN,NaN,19155.740234,NaN
2018-12-26,NaN,22878.449219,2467.699951,19327.060547,30.410000
2018-12-27,476.029999,23138.820312,2488.830078,20077.619141,29.959999
2018-12-28,484.170013,23062.400391,2485.739990,20014.769531,28.340000


In [5]:
# Create exchange rate ticker list based on initial investment data
currencies = list(set(['EUR' + i[1] + '=X' for i in investment.values()]))

# Remove EUR-EUR exchange rate
for currency in currencies:
    if currency[0:2] == currency[3:5]:
        currencies.remove(currency)
        
currencies = " ".join(currencies)

In [6]:
# Download exchange rate data
exchangerate_data = yf.download(currencies, start=start, end=end, groupby='ticker')
exchangerate = exchangerate_data['Adj Close']
exchangerate.tail()

[*********************100%***********************]  2 of 2 completed


,EURJPY=X,EURUSD=X
Date,,
2018-12-24,126.292000,1.137281
2018-12-25,125.769997,1.140394
2018-12-26,126.001999,1.141553
2018-12-27,126.404999,1.136131
2018-12-28,126.767998,1.143105


In [13]:
# Scrape EURIBOR rates

import csv
import requests
import io

year_range = int(end[0:4]) - int(start[0:4]) + 1
years = range(int(start[0:4]), int(start[0:4]) + year_range)

urls = ['https://www.emmi-benchmarks.eu/assets/components/rateisblue/file_processing/publication/processed/hist_EURIBOR_{}.csv'
        .format(i) for i in range(int(start[0:4]), int(start[0:4]) + year_range)]

for url in urls:
    s = requests.get(url).content
    if url == urls[0]:
        euribor = pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose()
    else:
        euribor = euribor.append(pd.read_csv(io.StringIO(s.decode('utf-8'))).transpose())
        
euribor3m = euribor[5]

In [11]:
euribor3m